In [1]:
import os
import sys
curr_dir=os.getcwd()
p=curr_dir.find('dev')
root=curr_dir[:p]
sys.path.append(root+'lib')
import psi4
import numpy as np
from P4toC4_aux import *
from SO_aux import SymOrbs

In [2]:
#BASIS='STO-3G'
#BASIS='def2-SV'
BASIS='CC-PVDZ'

In [3]:
psi4.set_memory('500 MB')
psi4.core.set_global_option("BASIS", BASIS)
psi4.core.set_global_option("SCF_TYPE", "pk")
psi4.core.set_global_option("REFERENCE", "RHF")
psi4.core.set_global_option("D_CONVERGENCE", 1e-8)
psi4.core.set_global_option("PUREAM", "True")
psi4.core.set_output_file('output.dat', False)

mol = psi4.geometry("""
0 1
N    0.00000000     0.12321896     0.00000000
H    1.79828758    -0.57068247     0.00000000
H   -0.89914379    -0.57068247     1.55736273
H   -0.89914379    -0.57068247    -1.55736273

no_reorient
units Bohr
""")

E, wf = psi4.energy('scf', return_wfn=True)
E

-56.194987542460474

In [4]:
basisset=wf.basisset()
#
#  C4_MO[p2c[i]] = P4_MO[i]     P4_MO[c2p[i]] = C4_MO[i]
#
p2c_map, p2c_scale = basis_mapping(basisset, verbose=0)
print(p2c_map)
print(np.round(p2c_scale,3))
naos=len(p2c_map)
c2p_map=np.zeros(naos, int)
#c2p_scale=np.zeros(naos)
for i in range(naos):
    c2p_map[p2c_map[i]] = i   
#c2p_map
# Print a new basis set in GENBAS format
#print(basisset.genbas())

[ 0  1  2  7  3  5  8  4  6  9 11 13 12 10 14 15 18 16 17 19 20 23 21 22
 24 25 28 26 27]
[1.    1.    1.    1.    1.    1.    1.    1.    1.    3.464 1.    1.
 2.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.   ]


In [5]:
mol=wf.molecule()
ptgr=mol.point_group()
print(f'{ptgr.symbol()}: order = {ptgr.order()}')
n_irrep=wf.nirrep()
g=wf.nmopi()
n_mo_pi=g.to_tuple()
print('MOs per irrep', n_mo_pi)

cs: order = 2
MOs per irrep (19, 10)


In [6]:
Ls=wf.aotoso()
print(Ls.shape)
# Psi4 MOs in SO basis
C_SO=wf.Ca()
#Cb=np.array(wf.Cb())
C_SO.shape

((29, 19), (29, 10))


((19, 19), (10, 10))

In [16]:
irrep_lst = []

for isym in range(ptgr.order()):
    print(f'\nSymmetry {isym}')
    SOs=SymOrbs(Ls.nph[isym], order=wf.nirrep())
    #SOs.print()
    print('Psi4  AO-order:', SOs.first_AOs())
    cfour_first_AOs = p2c_map[SOs.first_AOs()]
    ao_scale = p2c_scale[SOs.first_AOs()]
    print('AO scale      :', np.round(ao_scale,3))
    print('Cfour AO-order:', cfour_first_AOs)
    so_c2p = np.argsort(cfour_first_AOs)
    print('Cfour argsorted', so_c2p)
    nsos=len(so_c2p)
    so_p2c=np.zeros(nsos, int)
    for i in range(nsos):
        so_p2c[so_c2p[i]] = i
    print('And inverted   ', so_p2c)
    so_scale=SOs.inv_coef()
    scale = so_scale*ao_scale
    print('scale', np.round(scale,3))
    C=psi4_to_c4(C_SO.nph[isym], so_p2c, scale, use_scale=True)
    irrep_lst.append(C)
    
C_SOr = psi4.core.Matrix.from_array(irrep_lst)
C_SOr.shape


Symmetry 0
Psi4  AO-order: [ 0  1  2  4  5  7  8  9 12 13 14 15 17 18 19 20 21 22 23]
AO scale      : [1.    1.    1.    1.    1.    1.    1.    3.464 2.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.   ]
Cfour AO-order: [ 0  1  2  3  5  4  6  9 12 10 14 15 16 17 19 20 23 21 22]
Cfour argsorted [ 0  1  2  3  5  4  6  7  9  8 10 11 12 13 14 15 17 18 16]
And inverted    [ 0  1  2  3  5  4  6  7  9  8 10 11 12 13 14 15 18 16 17]
scale [1.    1.    1.    1.    1.    1.    1.    3.464 2.    1.    1.    1.
 1.    1.    1.414 1.414 1.414 1.414 1.414]

Symmetry 1
Psi4  AO-order: [ 3  6 10 11 16 19 20 21 22 23]
AO scale      : [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Cfour AO-order: [ 7  8 11 13 18 19 20 23 21 22]
Cfour argsorted [0 1 2 3 4 5 6 8 9 7]
And inverted    [0 1 2 3 4 5 6 9 7 8]
scale [1.    1.    1.    1.    1.    1.414 1.414 1.414 1.414 1.414]


((19, 19), (10, 10))

In [12]:
C4_cs = read_oldmos('OLDMOS.'+BASIS, n_mo_pi)

reading orbitals from OLDMOS.CC-PVDZ


In [17]:
sym=0
Corg=C_SO.nph[sym]
Creo=C_SOr.nph[sym]
Cc4=C4_cs[sym]
naos=n_mo_pi[sym]
mo=3
print('          Psi4    reordered    Cfour')
for k in range(naos):
    print(f'{k:3d}  {Corg[k,mo]:10.6f} {Creo[k,mo]:10.6f} {Cc4[k,mo]:10.6f}')
print(np.max(Creo[:,mo]-Cc4[:,mo]))

          Psi4    reordered    Cfour
  0    0.000557   0.000557   0.000557
  1    0.135227   0.135227   0.135227
  2    0.285136   0.285136   0.285136
  3    0.000000   0.000000   0.000000
  4    0.546099   0.000000   0.000000
  5    0.000000   0.546099   0.546099
  6    0.469090   0.469090   0.469090
  7    0.009419   0.002719   0.002719
  8    0.016314   0.000000   0.000000
  9    0.000000   0.008157   0.008157
 10   -0.091120  -0.091120  -0.091120
 11   -0.033749  -0.033749  -0.033749
 12    0.012583   0.012583   0.012583
 13    0.018589   0.018589   0.018589
 14   -0.128864  -0.091120  -0.091120
 15   -0.047728  -0.033749  -0.033749
 16    0.015411  -0.006292  -0.006292
 17   -0.008898   0.018589   0.018589
 18    0.026289   0.010897   0.010897
1.0713370135473355e-08


In [19]:
#
#  comparison Psi4-MOs and Cfour-MOs in their SO representation
#
for i in range(wf.nirrep()):
    print(np.max(abs(C_SOr.nph[i])-abs(C4_cs[i])))

1.0713370135473355e-08
7.79428599440024e-09


### Get symmetry ordered MOs in AO basis 

In [24]:
Ca_C4 = psi4_to_c4(Ca, mapping, scale)
Cb_C4 = psi4_to_c4(Cb, mapping, scale)

In [25]:
#
#  Write_oldmos needs an append option so it can be called for each irrep
#
#  Let's worry about UHF with symmetry later.
#
#
write_oldmos('PSIMOS', Ca_C4, Cbs=Cb_C4)